In [0]:
%run ./00_config

In [0]:
# --- 顧客マスタ (users) ---
users = """user_id,name,age,gender,email,region,registration_date,email_opt_in,last_updated,operation
U01,田中 太郎,25,Male,tanaka@example.com,東京,2024-01-10,true,2025-01-01 10:00:00,INSERT
U02,佐藤 次郎,45,Male,sato@example.com,千葉,2024-05-20,true,2025-01-01 10:00:00,INSERT
U03,鈴木 花子,65,Female,suzuki@example.com,神奈川,2024-11-15,true,2025-01-01 10:00:00,INSERT
"""

# --- 購買履歴 (transactions) ---
# 東エリアから届く初期データ
transactions_east = """order_id,user_id,product_id,amount,quantity,transaction_date
101,U01,P01,150000,1,2025-01-01 10:30:00
102,U02,P02,5000,2,2025-01-01 11:00:00
104,U01,P02,5000,1,2025-01-02 09:00:00
"""
# 西エリアから届く初期データ
transactions_west = """order_id,user_id,product_id,amount,quantity,transaction_date
103,U03,P03,35000,1,2025-01-02 15:00:00
"""

# --- 商品マスタ (products) ---
products = [
    ("P01", "ノートパソコン", 150000, "IT機器", "PC本体"),
    ("P02", "ワイヤレスマウス", 5000, "IT機器", "周辺機器"),
    ("P03", "ゲーミングモニター", 35000, "IT機器", "周辺機器"),
]
# Spark DataFrame を作成
products_df = spark.createDataFrame(
    products, 
    ["product_id", "product_name", "price", "category", "subcategory"]
)

In [0]:
import random
import string

def generate_unique_code(length=5):
    chars = string.ascii_lowercase + string.digits
    return ''.join(random.sample(chars, length))

random_suffix = generate_unique_code()

# 顧客マスタの書き出し (CSV)
dbutils.fs.put(f"{users_dir}users_{random_suffix}.csv", users, True)
print(f"{users_dir}users_{random_suffix}.csv")

# 購入履歴の書き出し (CSV)
dbutils.fs.put(f"{transactions_east_dir}transactions_east_{random_suffix}.csv", transactions_east, True)
print(f"{transactions_east_dir}transactions_east_{random_suffix}.csv")
dbutils.fs.put(f"{transactions_west_dir}transactions_west_{random_suffix}.csv", transactions_west, True)
print(f"{transactions_west_dir}transactions_west_{random_suffix}.csv")

# 商品マスタの書き出し (CSV) / 外部データソース
products_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(products_source_table)
print(products_source_table)

print("")
print("✅ 01_初期データ生成が完了しました。")